In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jul 27 01:40:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense ,LSTM, Dropout,Bidirectional,TimeDistributed
from sklearn.model_selection import train_test_split, KFold
from tensorflow import expand_dims
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from os.path import join
from tensorflow import constant
from time import sleep
from prophet import Prophet
import tensorflow as tf

#import FinanceDataReader as fdr

In [2]:
data=pd.read_csv('train.csv')

In [3]:
gl=pd.read_csv('goldlee.csv')

In [4]:
def convert_date(date_str):
  return pd.to_datetime(date_str, format="%Y%m%d").strftime("%Y-%m-%d")
data["일자"] =data["일자"].apply(convert_date)

In [5]:
def convert_date(date_str):
  return pd.to_datetime(date_str, format="%Y/%m/%d").strftime("%Y-%m-%d")
gl["일자"] =gl["일자"].apply(convert_date)

In [6]:
gl = gl[gl["일자"].isin(data["일자"].unique())]
gl.set_index('일자',inplace=True)

In [7]:
# prescaler
def getAOC(data):
  AOC = []
  close = data['종가']
  close_pre = close.shift(1)

  AOC.append(0)

  # aoc = (금일 종가 - 전일 종가) / 전일 종가
  for i in range(1, len(close)):
    aoc_value = ((close[i] - close_pre[i]) / close_pre[i]) * 100
    AOC.append(aoc_value)

  return AOC

#sort data by "종목명"
subject_Data=data.sort_values(by="종목코드",inplace=False)
subject_Data.head()
subject_Data.columns


pv_s_data=pd.pivot_table(subject_Data,values=['거래량', '시가', '고가', '저가', '종가'],index=['종목코드','일자'])
pv_s_data["AOC"] = getAOC(pv_s_data)

divided = pv_s_data.index.get_level_values('종목코드').unique()

dataframes = []

for i in divided:
    sliced_data = pv_s_data.loc[i]
    sliced_dataframe = sliced_data.reset_index()
    dataframes.append(sliced_dataframe)

sliced_dataframes=[]

for s in dataframes:
    end = s.pop('종가')
    sliced_dataframe = pd.merge(s, gl, left_on='일자', right_index=True, how='left')
    sliced_dataframe['종가'] = end
    sliced_dataframes.append(sliced_dataframe)

In [8]:
## sorting labels
labels=data[['종목코드','일자']]
labels = labels.sort_values(by=['종목코드','일자'])
labels=labels.drop('일자',axis=1)
labels=labels.drop_duplicates('종목코드')
data.sort_values(by=['종목코드',"일자"],inplace=True)

In [34]:
label_lis=[]
for label in labels:
  label_lis.append(label)

In [10]:
for sliced in sliced_dataframes:
  sliced.drop('일자',axis=1,inplace=True)

## Model shells

## TEST **Shell**

## Predict **Shell**

In [12]:
%pwd

'c:\\Users\\admin\\Downloads'

In [13]:
%cd /content/drive/MyDrive/2023_1st_vac/KRX_modelings

c:\content\drive\MyDrive\2023_1st_vac\KRX_modelings


In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense ,LSTM, Dropout,Bidirectional,TimeDistributed
from sklearn.model_selection import train_test_split, KFold
from tensorflow import expand_dims
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from os.path import join
from tensorflow import constant
from time import sleep
from prophet import Prophet
import tensorflow as tf


class LSTM2():
  def __init__(self,data,idx,day):
    self.data=data
    self.idx=idx
    self.time_steps=60
    self.day=day

  def create_sequences(self, data):
    num_samples, num_features = data.shape
    sequences = []
    for i in range(num_samples - self.time_steps +1):
      sequences.append(data[i:i + self.time_steps, :])
    return np.array(sequences)

  def inverse_sequences(self,data_sequences):
    num_samples, num_steps, num_features = data_sequences.shape
    data = np.zeros((num_samples + self.time_steps - 1, num_features))
    for i in range(num_samples):
      data[i:i + self.time_steps, :] += data_sequences[i, :, :]
    data /= self.time_steps
    return data[:num_samples]
    #frquency domain inverse laplace transform

  def slicing_data(self):
    self.x_scaler = MinMaxScaler()
    self.y_scaler = MinMaxScaler()
    # Scaling features
    x_data = self.data[self.data.columns[:]]
    x_data = self.x_scaler.fit_transform(x_data)
    # Scaling target variable
    y_data = self.data[self.data.columns[:]]
    y_data = self.y_scaler.fit_transform(y_data)
    # validation set
    x_train_p, x_test, y_train_p, y_test = train_test_split(x_data, y_data, test_size=0.2, shuffle=False)
    x_train, x_val, y_train, y_val = train_test_split(x_train_p, y_train_p, test_size=0.25, shuffle=False)
    __t,x_final,__t,__t=train_test_split(x_data, y_data, test_size=60/len(y_data), shuffle=False)

    self.num_features = x_data.shape[1]
    self.num_samples=x_data.shape[0]

    self.x_train = self.create_sequences(x_train)
    self.x_test = self.create_sequences(x_test)
    self.x_val = self.create_sequences(x_val)
    self.x_final = self.create_sequences(x_final)

    self.y_train=self.create_sequences(y_train)
    self.y_test=self.create_sequences(y_test)
    self.y_val=self.create_sequences(y_val)

  def model_struct(self):
    self.model = Sequential()
    # layers
    self.model.add(Bidirectional(LSTM(1024, return_sequences=True, input_shape=(self.time_steps, self.num_features))))
    self.model.add(Dense(256, activation='relu'))
    self.model.add(Dense(128, activation='relu'))
    self.model.add(Dense(64, activation='relu'))
    self.model.add(Dense(32, activation='relu'))
    self.model.add(Dense(16, activation='relu'))
    # output layer (many-to-many with TimeDistributed)
    self.model.add(TimeDistributed(Dense(len(self.data.columns), activation='relu')))
  
  def train_model(self):
    model_save_path = '/content/drive/MyDrive/2023_1st_vac/KRX_modelings/best_model/LSTM2/{}/'.format(self.day)
    self.filename = join(model_save_path, 'checkpoint_{}_{}.ckpt'.format(self.day,self.idx))
    checkpoint = ModelCheckpoint(self.filename, save_weights_only=True, save_best_only=True, monitor='val_loss', verbose=0)
    earlystopping = EarlyStopping(monitor='val_loss', patience=100)
    self.model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
    self.history = self.model.fit(self.x_train, self.y_train, epochs=1, batch_size=128, validation_data=(self.x_val, self.y_val), shuffle=False, callbacks=[checkpoint, earlystopping], verbose=0)


  def get_gap_by_test(self):
    self.model.load_weights(self.filename)
    pred = self.model.predict(self.x_test,verbose=0)
    rescaled_pred = self.y_scaler.inverse_transform(self.inverse_sequences(pred))
    rescaled_real = self.y_scaler.inverse_transform(self.inverse_sequences(self.y_test))

    gaps = []

    for p, r in zip(rescaled_pred, rescaled_real):
        gap = np.abs(p - r)
        gaps.append(gap)

    avg_gap = np.mean(gaps)

    return avg_gap



  def return_val(self):
    self.slicing_data()
    self.model_struct()
    self.train_model()
    self.model.load_weights(self.filename)

    next_input_data = np.copy(self.x_final)
    for day in range(1, 16):
        pred_day = self.model.predict(np.expand_dims(next_input_data[-1], axis=0),verbose=0)
        next_input_data = np.concatenate((next_input_data, pred_day), axis=0)

    next_15_days_data = next_input_data[-15:]
    next_15_days_data=self.y_scaler.inverse_transform(self.inverse_sequences(next_15_days_data))

    columns=self.data.columns.tolist()
    ret_df = pd.DataFrame(next_15_days_data, columns=columns)

    prices=[]
    for end_price in ret_df.iloc[:,-1]:
      prices.append(end_price)

    gap=self.get_gap_by_test()

    return prices,gap


In [24]:
idx=0

pred_list=[]
gap_acc=[]
out_labels=[]

In [25]:
for c in sliced_dataframes:
  print(labels.iloc[idx])
  print("{}번째 종목코드".format(idx))
  LSTM_model=LSTM2(c,idx=idx,day='0727')
  pred,gap = LSTM_model.return_val()
  sleep(0.5)
  pred_list.append(pred)
  gap_acc.append(gap)
  idx+=1
  #to prevent model corrupting

종목코드    A000020
Name: 502, dtype: object
0번째 종목코드
종목코드    A000040
Name: 118, dtype: object
1번째 종목코드
종목코드    A000050
Name: 243, dtype: object
2번째 종목코드
종목코드    A000070
Name: 760, dtype: object
3번째 종목코드
종목코드    A000080
Name: 1803, dtype: object
4번째 종목코드
종목코드    A000100
Name: 1328, dtype: object
5번째 종목코드
종목코드    A000120
Name: 16, dtype: object
6번째 종목코드
종목코드    A000140
Name: 1804, dtype: object
7번째 종목코드
종목코드    A000150
Name: 503, dtype: object
8번째 종목코드
종목코드    A000180
Name: 840, dtype: object
9번째 종목코드
종목코드    A000210
Name: 33, dtype: object
10번째 종목코드
종목코드    A000220
Name: 1320, dtype: object
11번째 종목코드
종목코드    A000230
Name: 1412, dtype: object
12번째 종목코드
종목코드    A000240
Name: 1823, dtype: object
13번째 종목코드
종목코드    A000250
Name: 776, dtype: object
14번째 종목코드
종목코드    A000270
Name: 298, dtype: object
15번째 종목코드
종목코드    A000300
Name: 416, dtype: object
16번째 종목코드
종목코드    A000320
Name: 346, dtype: object
17번째 종목코드
종목코드    A000370
Name: 1903, dtype: object
18번째 종목코드
종목코드    A000390
Name: 787, dtype: ob

In [37]:
label_lis = labels['종목코드']

In [48]:
pred_15 = []
for i in pred_list:
    pred_15.append(i[-1])

In [50]:
past_list = []
for i in sliced_dataframes:
  past_list.append(i['종가'][493])

In [66]:
def subtract_arrays(arr1, arr2):
    if len(arr1) != len(arr2):
        raise ValueError("Arrays must have the same length")

    result = []
    for i in range(len(arr1)):
        result.append(arr1[i] - arr2[i])
    return result

def divide_arrays(arr1, arr2):
    if len(arr1) != len(arr2):
        raise ValueError("Arrays must have the same length")

    result = []
    for i in range(len(arr1)):
        result.append(arr1[i] / arr2[i])
    return result

In [80]:
# 수익률
benefit = divide_arrays(pred_15, past_list)

# 무위험지표금리
# 자금 차입자의 신용위험 및 자금거래의 유덩성 위험이 배제된 상태에서
# 자금의 수요와 공급에 의해 결정되는 금융기관의 평균적 자금조달비용

risk_free_rate = 3.449 #KOFR에서 실시간으로 찾아봄
sharpes = []
for i in range(len(benefit)):
    sharpes.append((benefit[i] - risk_free_rate) / np.std(benefit))

In [83]:
pred_csv_df=pd.DataFrame({
    '종목코드': label_lis,
    '샤프지수': sharpes
})

In [84]:
pred_csv_df = pred_csv_df.sort_values(by='샤프지수', ascending = False)

In [85]:
pred_csv_df.drop(columns=['샤프지수'], inplace = True)

In [86]:
pred_csv_df['순위'] = list(range(1, 2001))

In [87]:
pred_csv_df.to_csv('{}.csv'.format("230728_v0.3"),index=False)